# Graph File Pre-Processing

Take ROOT files turn them into point clouds which can then be turned into graphs


In [1]:
import numpy as np

import pandas as pd

from operator import truth
import pandas as pd
import numpy as np
import awkward as ak

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn
import uproot
import torch

from tqdm import tqdm
import timeit
import os

import dill

In [2]:
# take ROOT file and convert to an awkward array
def fileToAwk(path):
    file = uproot.open(path)
    tree = file['tree']
    
    awk = tree.arrays(tree.keys())
    return awk

input_features = ["part_px", "part_py", "part_pz", "part_energy",
                  "part_deta", "part_dphi", "part_d0val", "part_d0err", 
                  "part_dzval", "part_dzerr", "part_isChargedHadron", "part_isNeutralHadron", 
                  "part_isPhoton", "part_isElectron", "part_isMuon" ] # features used to train the model

 
# take AWK dict and convert to a point cloud
def awkToPointCloud(awkDict, input_features):
    available_features = awkDict.type.keys() # all features

    featureVector = []
    for jet in tqdm(range(len(awkDict)), total=len(awkDict)):
        currJet = awkDict[jet][input_features]
        pT = np.array(np.sqrt(currJet['part_px'] ** 2 + currJet['part_py'] ** 2))
        # creates numpy array to represent the 4 momenta of all particles in a jet
        currJet = np.column_stack((np.array(currJet['part_px']), np.array(currJet['part_py']), 
                                   np.array(currJet['part_pz']), np.array(currJet['part_energy']), pT
                                   , np.array(currJet['part_deta']), np.array(currJet['part_dphi']), 
                                   np.array(currJet["part_d0val"]), np.array(currJet["part_d0err"]), 
                                   np.array(currJet["part_dzval"]), np.array(currJet["part_dzerr"]), 
                                   np.array(currJet["part_isChargedHadron"]), np.array(currJet["part_isNeutralHadron"]), 
                                   np.array(currJet["part_isPhoton"]), np.array(currJet["part_isElectron"]), 
                                   np.array(currJet["part_isMuon"])))
        
        featureVector.append(currJet)
    return np.array(featureVector)

In [3]:
from scipy.spatial import cKDTree

#take point cloud and build KNN graph
def buildKNNGraph(points, k):
    
    # Compute k-nearest neighbors
    tree = cKDTree(points)
    dists, indices = tree.query(points, k+1)  # +1 to exclude self
    
    # Build adjacency matrix
    num_points = len(points)
    adj_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j in indices[i, 1:]:  # exclude self
            adj_matrix[i, j] = 1
            adj_matrix[j, i] = 1
    return adj_matrix

# take adjacency matrix and turn it into a DGL graph
def adjacencyToDGL(adj_matrix):
    adj_matrix = sp.coo_matrix(adj_matrix)
    g_dgl = dgl.from_scipy(adj_matrix)
        
    return g_dgl

In [4]:
import scipy.sparse as sp
import dgl
import pickle

# wrap the functionality of fileToAwk and awkToPointCloud in a function to return a point cloud numpy array
def fileToPointCloudArray(jetType, input_features):
    filepath = f'/Volumes/Yash SSD/JetClass/JetRoots/{jetType}_000.root' # original root file
    savepath = f'/Volumes/Yash SSD/JetClass/PointClouds/{jetType}.npy' # save file
    awk = fileToAwk(filepath)
    nparr = awkToPointCloud(awk, input_features)
    
    return nparr

# wrap the functionality of fileToPointCloudArray and the 
def fileToGraph(jetType, k=3, save=True):
    print(f'Starting processing on {jetType} jets')
    pointCloudArr = fileToPointCloudArray(jetType, input_features)
    
    saveFilePath = f'/Volumes/Yash SSD/Multi Level Jet Tagging/{jetType}.pkl'
    
    savedGraphs = []
    for idx, pointCloud in tqdm(enumerate(pointCloudArr), leave=False, total=len(pointCloudArr)):
        try:
            adj_matrix = buildKNNGraph(pointCloud, k)
            graph = adjacencyToDGL(adj_matrix)
            
            graph.ndata['feat'] = torch.tensor(pointCloud, dtype=torch.float32)
            
            savedGraphs.append(graph)
            
            del adj_matrix, graph
        except Exception as e:
            print(e)
            
    
    if save:
        with open(saveFilePath, 'wb') as f:
            pickle.dump(savedGraphs, f, protocol=pickle.HIGHEST_PROTOCOL)
        
        
        del pointCloudArr, savedGraphs
        
    print(f'Graphs for {jetType} processing complete!')
        
    return savedGraphs

def groupToGraph(jetTypeList, groupName):
    allGraphs = []
    for jetType in jetTypeList:
        allGraphs += fileToGraph(jetType, save=False)
    
    saveFilePath = f'/Volumes/Yash SSD/Multi Level Jet Tagging/{groupName}.pkl' 
    return allGraphs

In [6]:
# process all jetTypes
Higgs = ['HToBB', 'HToCC', 'HToGG', 'HToWW2Q1L', 'HToWW4Q']
Vector = ['WToQQ', 'ZToQQ']
Top = ['TTBar', 'TTBarLep']
QCD = ['ZJetsToNuNu']
Emitter = ['Emitter-Vector', 'Emitter-Top', 'Emitter-Higgs', 'Emitter-QCD']
allJets = Higgs + Vector + Top + QCD

#for jetType in allJets:
#    fileToGraph(jetType)

allGraphs = groupToGraph(Higgs, "Emitter-Higgs")

Starting processing on HToBB jets


100%|██████████████████████████████████| 100000/100000 [02:50<00:00, 586.30it/s]
/var/folders/0q/bs1s8dl52_g396mh_5rg0zb40000gn/T/ipykernel_940/2564396636.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(featureVector)


Graphs for HToBB processing complete!
Starting processing on HToCC jets


100%|██████████████████████████████████| 100000/100000 [02:49<00:00, 590.68it/s]


Graphs for HToCC processing complete!
Starting processing on HToGG jets


...

 92%|███████████████████████████████▏  | 91757/100000 [00:19<00:01, 5824.21it/s]

index 3 is out of bounds for axis 1 with size 3


Graphs for HToWW2Q1L processing complete!
Starting processing on HToWW4Q jets


100%|██████████████████████████████████| 100000/100000 [02:52<00:00, 580.67it/s]
                                                                                

Graphs for HToWW4Q processing complete!


In [ ]:
with open(f'/Volumes/Yash SSD/Multi Level Jet Tagging/Emitter-Higgs.pkl', 'wb') as f:
    pickle.dump(allGraphs, f)
    
print("DONE")